In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm

# Capstone Project - Function 6

## Function 6 - Cake Recipe

Time to get cooking! You are optimising a cake recipe. There are five ingredients. The outputs correspond to the sum of different objectives: flavor, consistency, calories, waste and cost. Each objective receives negative points by our expert taster. You want this sum to be as close to zero as possible!

### 1.Loading the available data:

In [10]:
# Loading the baseline data
X = np.load('initial_data/function_6/initial_inputs.npy')
Y = np.load('initial_data/function_6/initial_outputs.npy')
X2 = np.load('initial_data2/function_6/initial_inputs.npy')
Y2 = np.load('initial_data2/function_6/initial_outputs.npy')

X = np.concatenate((X,X2), dtype = float)
Y = np.concatenate((Y,Y2), dtype = float)

# Precision notation
np.set_printoptions(suppress=False)

# Adding knwon inputs/outputs from previous weeks
Xn = np.array([
    [0.473684, 0.421052, 0.315789, 0.842105, 0.263157],
    [0.842105, 0.052631, 0.894736, 0, 0.947368],
    [0.553684, 0.501052, 0.385789, 0.742105, 0.463157],
    [0.742105, 0.452631, 0.294736, 0.338222, 0.947368],
    [0.684210, 0.105263, 0.789473, 0.684210, 0.000001],
    [0.789473, 0.210526, 0.68421,  0.68421,  0.105263],
    [0.000001, 0.999999, 0.999999, 0.999999, 0.999999],
    [0.583333, 0.375,    0.583333, 0.625,    0.125   ],
    [0.791666, 0.25, 0.666666, 0.666666, 0.125   ],
    [0.666666, 0.958333, 0.416666, 0.333333, 0.625   ],
    [7.91666e-01, 1.00000e-06, 1.25000e-01, 4.58333e-01, 1.66666e-01],
    [1.00000e-06, 2.91666e-01, 9.99999e-01, 9.99999e-01, 2.08333e-01],
    [1.00000e-06, 2.91666e-01, 7.50000e-01, 6.25000e-01, 1.66666e-01],
    [0.708333, 0.291666, 0.625,    0.666667, 0.041666],
    [8.33330e-02, 1.00000e-06, 7.91666e-01, 4.16660e-02, 6.25000e-01],
    [0.291666, 0.25,     0.958333, 0.916666, 0.041666],
    [0.708333, 0.291666, 0.625,    0.625,    0.083333],
    [0.709433, 0.301666, 0.612243, 0.624,    0.073333],
    [0.37931,  0.068965, 0.999998, 0.551724, 0.137931]
    ])
X = np.concatenate((X,Xn), dtype = float)

Yn = np.array(
[
    -0.6916235125,
    -2.547480536,
    -0.8906737471,
    -2.064696855,
    -0.8324098332,
    -0.7382412055,
    -2.436658783,
    -0.4567739676,
    -0.5276128894,
    -2.107046929,
    -1.61850582,
    -1.080043811,
    -0.8067561148,
    -0.5416242645162979,
    -2.060325708048024,
    -0.7568987484,
    -0.4970914611,
    -0.5180188044,
    -0.9369391047
])
Y = np.concatenate((Y,Yn), dtype = float)


In [4]:
X

array([[7.28186105e-01, 1.54692570e-01, 7.32551669e-01, 6.93996509e-01,
        5.64013105e-02],
       [2.42384347e-01, 8.44099972e-01, 5.77809099e-01, 6.79021284e-01,
        5.01952888e-01],
       [7.29522610e-01, 7.48106200e-01, 6.79774641e-01, 3.56552279e-01,
        6.71053683e-01],
       [7.70620242e-01, 1.14403744e-01, 4.67799319e-02, 6.48324285e-01,
        2.73549053e-01],
       [6.18812299e-01, 3.31802137e-01, 1.87287868e-01, 7.56238474e-01,
        3.28834798e-01],
       [7.84958094e-01, 9.10682349e-01, 7.08120104e-01, 9.59225429e-01,
        4.91149586e-03],
       [1.45110786e-01, 8.96684598e-01, 8.96322235e-01, 7.26271537e-01,
        2.36271991e-01],
       [9.45069068e-01, 2.88459051e-01, 9.78805764e-01, 9.61655587e-01,
        5.98015936e-01],
       [1.25720155e-01, 8.62724692e-01, 2.85443322e-02, 2.46605272e-01,
        7.51206241e-01],
       [7.57594355e-01, 3.55831415e-01, 1.65228997e-02, 4.34207205e-01,
        1.12433044e-01],
       [5.36796903e-01, 3.0878

In [5]:
Y

array([-0.71426495, -1.20995524, -1.67219994, -1.53605771, -0.82923655,
       -1.24704893, -1.23378638, -1.69434344, -2.57116963, -1.30911635,
       -1.14478485, -1.91267714, -1.62283895, -1.35668211, -2.0184254 ,
       -1.70255784, -1.29424696, -0.93575656, -2.15576776, -1.74688209,
       -1.51269542, -1.34959856, -1.82739562, -1.71602331, -0.97248425,
       -1.20521138, -1.10099104, -1.91309535, -1.05157417, -1.19458833,
       -2.06906505, -1.48937327, -1.82528398, -1.7476171 , -1.35823203,
       -1.44913772, -1.81134837, -2.00771782, -0.99967761, -1.67533648,
       -0.69162351, -2.54748054, -0.89067375, -2.06469686, -0.83240983,
       -0.73824121, -2.43665878, -0.45677397, -0.52761289, -2.10704693,
       -1.61850582, -1.08004381, -0.80675611, -0.54162426, -2.06032571,
       -0.75689875, -0.49709146])

In [11]:
y_max = np.max(Y)
print("Max output until now: ", y_max, "which corresponds to input: ", X[np.where(Y == y_max)][0])

Max output until now:  -0.4567739676 which corresponds to input:  [0.583333 0.375    0.583333 0.625    0.125   ]


### 2. Build a Bayesian Model

In [12]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define the custom kernel (RBF with anisotropic length scales)
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=[1.0, 0.5, 2.0, 0.8, 1.5])

# Build a GP model using the Gaussian kernel
gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=11)
gpr.fit(X, Y)

GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=[1, 0.5, 2, 0.8, 1.5]),
                         n_restarts_optimizer=11)

### 3. Acquisition Function 1 - Upper Confidence Bound

Upper-Confidence Bound action selection uses uncertainty in the action-value estimates for balancing exploration and exploitation. Since there is inherent uncertainty in the accuracy of the action-value estimates when we use a sampled set of rewards thus UCB uses uncertainty in the estimates to drive exploration.

In [13]:
import itertools as it

x1 = np.linspace(0, 1, 30)

dim = 5
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

# We keep exploring more than exploiting as we did not had satisfying results yet for F6
ucb1 = mean + 1.56 * std

idx_max = np.argmax(ucb1)
next_query = X_grid[idx_max]
print("UCB - Next Query - Idea 1: ", next_query)

UCB - Next Query - Idea 1:  [0.         0.         0.24137931 1.         0.        ]


### 4. Acquisition Function 2 - Probability of Improvement

In [14]:
from scipy.stats import norm

# Current best observed value (maximum)
f_best = np.max(Y)

# Compute PI values for candidate points
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

# Choose the next point with maximum PI value
next_idx = np.argmax(pi_values)
next_query = X_grid[next_idx]

print("PI - Next Query - Idea 1: ", next_query)

PI - Next Query - Idea 1:  [0.79310345 0.27586207 0.65517241 0.65517241 0.13793103]


### 5. Acquisition Function - Thompson Sampling

In [7]:
# Compute Thompson Sampling values for candidate points
def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

# Choose the next point with maximum Thompson Sampling value
next_idx = np.argmax(thompson_values)
next_query = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", next_query)


Thompson Sampling - Next Query:  [0.29166667 0.25       0.95833333 0.91666667 0.04166667]


### 6. Acquisition Function - Bayesian Expected Losses

In [8]:
# Compute the expected loss for candidate points
def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - y_max) / sigma
    expected_loss = (mu - y_max) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

# We re-use X_grid due to computational needs
expected_loss_values = [compute_expected_loss(x) for x in X_grid]

# Choose the next point with minimum expected loss
next_idx = np.argmin(expected_loss_values)
next_query = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", next_query)


KeyboardInterrupt



### 7. Acquisition Function - Expected Improvement

In [23]:
# Compute Expected Improvement for candidate points
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

# Choose the next point with maximum EI value
next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)


Expected Improvement - Next Query:  [0.78947368 0.         0.         0.         0.        ]


In [27]:
X_test = [[0.82352941, 0., 0., 0.5,  0.]]
formatted_UCB = "{}{:.6f}-{:.6f} -> {}".format("UCB: ",X_test[0][0],X_test[0][1], gpr.predict(X_test))
X_test = [[0.79166667, 0., 0.125, 0.45833333, 0.16666667]]
formatted_PI = "{}{:.6f}-{:.6f} -> {}".format("PI: ",X_test[0][0],X_test[0][1], gpr.predict(X_test))
X_test = [[0.66666667, 0.95833333, 0.41666667, 0.33333333, 0.625]]
formatted_TS = "{}{:.6f}-{:.6f} -> {}".format("TS: ",X_test[0][0],X_test[0][1], gpr.predict(X_test))
#X_test = [[0.12820513, 0.15384615, 0.76923077, 0.17948718]]
#formatted_BL = "{}{:.6f}-{:.6f} -> {}".format("TS: ",X_test[0][0],X_test[0][1], gpr.predict(X_test))
X_test = [[0.78947368, 0.,         0.,         0.,         0.,        ]]
formatted_EI = "{}{:.6f}-{:.6f} -> {}".format("EI: ",X_test[0][0],X_test[0][1], gpr.predict(X_test))

print(formatted_UCB)
print(formatted_PI)
print(formatted_TS)
#print(formatted_BL)
print(formatted_EI)

UCB: 0.823529-0.000000 -> [0.43104126]
PI: 0.791667-0.000000 -> [-0.03587611]
TS: 0.666667-0.958333 -> [-0.20236792]
EI: 0.789474-0.000000 -> [0.51858942]
